In [1]:
from agentex import Agentex

client = Agentex(base_url="http://localhost:5003")

In [2]:
AGENT_NAME = "at010-agent-chat"

In [3]:
# (REQUIRED) Create a new task. For Agentic agents, you must create a task for messages to be associated with.
import uuid

rpc_response = client.agents.create_task(
    agent_name=AGENT_NAME,
    params={
        "name": f"{str(uuid.uuid4())[:8]}-task",
        "params": {}
    }
)

task = rpc_response.result
print(task)

Task(id='2cb5e5b7-b8ba-4483-b7d3-635e6524e6d1', created_at=datetime.datetime(2025, 8, 27, 19, 1, 16, 906850, tzinfo=TzInfo(UTC)), name='4f5599a2-task', params={}, status='RUNNING', status_reason='Task created, forwarding to ACP server', updated_at=datetime.datetime(2025, 8, 27, 19, 1, 16, 906850, tzinfo=TzInfo(UTC)))


In [4]:
# Send an event to the agent

# The response is expected to be a list of TaskMessage objects, which is a union of the following types:
# - TextContent: A message with just text content   
# - DataContent: A message with JSON-serializable data content
# - ToolRequestContent: A message with a tool request, which contains a JSON-serializable request to call a tool
# - ToolResponseContent: A message with a tool response, which contains response object from a tool call in its content
# - ReasoningContent: A message with a reasoning content, which contains a reasoning object from a tool call in its content

# When processing the message/send response, if you are expecting more than TextContent, such as DataContent, ToolRequestContent, or ToolResponseContent, you can process them as well

rpc_response = client.agents.send_event(
    agent_name=AGENT_NAME,
    params={
        "content": {"type": "text", "author": "user", "content": "Find me a recipe on spaghetti"},
        "task_id": task.id,
    }
)

event = rpc_response.result
print(event)

Event(id='edc2171e-0a9c-4ace-9ed5-5d85e92130ee', agent_id='a1abb90e-c673-4448-a4e2-841170568840', sequence_id=1836, task_id='2cb5e5b7-b8ba-4483-b7d3-635e6524e6d1', content=TextContent(author='user', content='Find me a recipe on spaghetti', attachments=None, format='plain', style='static', type='text'), created_at=datetime.datetime(2025, 8, 27, 19, 1, 16, 940148, tzinfo=TzInfo(UTC)))


In [5]:
# Subscribe to the async task messages produced by the agent
from agentex.lib.utils.dev_tools import subscribe_to_async_task_messages

task_messages = subscribe_to_async_task_messages(
    client=client,
    task=task, 
    only_after_timestamp=event.created_at, 
    print_messages=True,
    rich_print=True,
    timeout=60,
)

╭───────────────────────── USER [08/27/2025 19:01:17] ─────────────────────────╮
│ Find me a recipe on spaghetti                                                │
╰──────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────── AGENT [08/27/2025 19:01:18] ─────────────────────────╮
│ I'm sorry, but I cannot process messages about spaghetti. This guardrail was │
│ put in place for demonstration purposes. Please ask me about something else! │
╰──────────────────────────────────────────────────────────────────────────────╯

Streaming timed out after 60 seconds - returning collected messages


In [6]:
rpc_response = client.agents.create_task(
    agent_name=AGENT_NAME,
    params={
        "name": f"{str(uuid.uuid4())[:8]}-task",
        "params": {}
    }
)

task = rpc_response.result
print(task)

Task(id='6495fc1f-3775-4d84-9406-c2953a07e36d', created_at=datetime.datetime(2025, 8, 27, 19, 2, 18, 331332, tzinfo=TzInfo(UTC)), name='a6c25875-task', params={}, status='RUNNING', status_reason='Task created, forwarding to ACP server', updated_at=datetime.datetime(2025, 8, 27, 19, 2, 18, 331332, tzinfo=TzInfo(UTC)))


In [7]:
rpc_response = client.agents.send_event(
    agent_name=AGENT_NAME,
    params={
        "content": {"type": "text", "author": "user", "content": "Find me the factorial of 31, think about how you will do it."},
        "task_id": task.id,
    }
)

event = rpc_response.result
print(event)

Event(id='aaebf675-11b5-4d7e-b2c9-0574798b6dec', agent_id='a1abb90e-c673-4448-a4e2-841170568840', sequence_id=1840, task_id='6495fc1f-3775-4d84-9406-c2953a07e36d', content=TextContent(author='user', content='Find me the factorial of 31, think about how you will do it.', attachments=None, format='plain', style='static', type='text'), created_at=datetime.datetime(2025, 8, 27, 19, 2, 18, 361054, tzinfo=TzInfo(UTC)))


In [8]:
# Subscribe to the async task messages produced by the agent
from agentex.lib.utils.dev_tools import subscribe_to_async_task_messages

task_messages = subscribe_to_async_task_messages(
    client=client,
    task=task, 
    only_after_timestamp=event.created_at, 
    print_messages=True,
    rich_print=True,
    timeout=60,
)

╭───────────────────────── USER [08/27/2025 19:02:18] ─────────────────────────╮
│ Find me the factorial of 31, think about how you will do it.                 │
╰──────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────── AGENT [08/27/2025 19:02:25] ─────────────────────────╮
│ 🧠 Reasoning                                                                 │
│                                                                              │
│ Considering factorial calculation                                            │
│                                                                              │
│ The user asked for the factorial of 31, which is a bit tricky. I could use   │
│ the calculator tool, but it only handles two-number arithmetic, while        │
│ calculating a factorial involves multiple multiplications. I think I might   │
│ need to rely on an iterative method or do the math manually. I remember the  │
│ values of smaller factorials like 20! and 21! but should compute carefully   │
│ to ensure accuracy. Let's figure out the best method to obtain the correct   │
│ result.                                                                      │
│                                                                              │
│ Planning factorial computation                                               │
│                                                                              │
│ The calculator only accepts two numbers at a time, but I can use iterative   │
│ multiplication to compute the factorial step by step. I should use the       │
│ calculator repeatedly, which is fine as long as I keep it all clear and      │
│ avoid showing my thought process. I can describe my method as multiplying    │
│ integers from 1 to 31, which is a straightforward algorithm. I’ll also refer │
│ to known factorial values, like 10! and 20!, to ensure I get 31! correct.    │
│                                                                              │
│ Calculating stepwise factorial                                               │
│                                                                              │
│ I can compute the factorial step by step using functions.calculator, which   │
│ will help ensure accuracy. I need to be careful to avoid revealing my        │
│ thought process too much because the calculator will return just the         │
│ results. I’ll start at 1 and multiply by each integer up to 31, which means  │
│ I’ll make around 30 calls. I can definitely proceed by sequentially calling  │
│ the calculator to get the values I need. Let's get started with this         │
│ approach!                                                                    │
╰──────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────── AGENT [08/27/2025 19:02:35] ─────────────────────────╮
│ 🔧 Tool Request: calculator                                                  │
│                                                                              │
│ Arguments:                                                                   │
│                                                                              │
│                                                                              │
│  {                                                                           │
│    "a": 1,                                                                   │
│    "b": 2,                                                                   │
│    "operation": "multiply"                                                   │
│  }                                                                           │
│                                                                              │
╰──────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────── AGENT [08/27/2025 19:02:35] ─────────────────────────╮
│ ✅ Tool Response: calculator                                                 │
│                                                                              │
│ The result of 1.0 multiply 2.0 is 2                                          │
╰──────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────── AGENT [08/27/2025 19:02:38] ─────────────────────────╮
│ 🔧 Tool Request: calculator                                                  │
│                                                                              │
│ Arguments:                                                                   │
│                                                                              │
│                                                                              │
│  {                                                                           │
│    "a": 2,                                                                   │
│    "b": 3,                                                                   │
│    "operation": "multiply"                                                   │
│  }                                                                           │
│                                                                              │
╰──────────────────────────────────────────────────────────────────────────────╯

...

╭──────────────────────── AGENT [08/27/2025 19:02:48] ─────────────────────────╮
│ ✅ Tool Response: calculator                                                 │
│                                                                              │
│ The result of 40320.0 multiply 9.0 is 362880                                 │
╰──────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────── AGENT [08/27/2025 19:02:51] ─────────────────────────╮
│ 🔧 Tool Request: calculator                                                  │
│                                                                              │
│ Arguments:                                                                   │
│                                                                              │
│                                                                              │
│  {                                                                           │
│    "a": 362880,                                                              │
│    "b": 10,                                                                  │
│    "operation": "multiply"                                                   │
│  }                                                                           │
│                                                                              │
╰──────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────── AGENT [08/27/2025 19:02:51] ─────────────────────────╮
│ ✅ Tool Response: calculator                                                 │
│                                                                              │
│ The result of 362880.0 multiply 10.0 is 3628800                              │
╰──────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────── AGENT [08/27/2025 19:02:53] ─────────────────────────╮
│ 🔧 Tool Request: calculator                                                  │
│                                                                              │
│ Arguments:                                                                   │
│                                                                              │
│                                                                              │
│  {                                                                           │
│    "a": 3628800,                                                             │
│    "b": 11,                                                                  │
│    "operation": "multiply"                                                   │
│  }                                                                           │
│                                                                              │
╰──────────────────────────────────────────────────────────────────────────────╯

Streaming timed out after 60 seconds - returning collected messages
